In [1]:
import sys
import pandas as pd
from lxml import etree
import re
from unidecode import unidecode

print(sys.executable)

/opt/anaconda3/envs/pouille/bin/python


# Preprocessing

1. extration des articles à lier et des features utiles dans df
2. chargement du référentiel des communes (COG 2011) dans le df main_insee_commune

In [2]:
# Extraction des features de chaque article à lier dans un df

xml_file = '../../data/PO_t11/PO_t11.xml'
xml_tree = etree.parse(xml_file)

data = []

i = 0 # compter les articles

for article in xml_tree.xpath('//article'):
    i+=1
    # on ne retient pas les lieux hors France
    if article.find('./localisationpa') is not None:
        continue
    vedettes = article.xpath('./vedette/i')
    # on ne retient que la première vedette
    vedette = vedettes[0].text if vedettes else ''

    # on ne retient que le premier dpt de localisation
    localisationde = article.find('./localisationde')
    localisationde = localisationde.text if localisationde is not None else 'none'

    # on ne retient que le premier canton de localisation
    localisationca = article.find('./localisationca')
    if localisationca is not None:
        localisationca = re.search(
            r'<localisationca>(.*?)<\/localisationca>',
            etree.tostring(localisationca, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        '''
        localisationca = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca)
    else:
        localisationca = 'none'

    #  on ne retient que la première commune de localisation
    localisationco = article.find('./localisationco')
    if localisationco is not None:
        localisationco = re.search(
            r'<localisationco>(.*?)<\/localisationco>',
            etree.tostring(localisationco, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        - 'c<sup>nes</sup> de '
        - 'c<sup>nes</sup> d’'
            '''
        localisationco = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco)

    else:
        localisationco = 'none'

    # Extraire le contenu textuel de la balise "article"
    article_text = re.sub(r'\W', ' ', article.xpath('string(.)'))

    # on insère tout dans un dictionnaire
    data.append({
        'old-id': article.get('old-id'),
        'vedette': vedette,
        'localisationde': localisationde,
        'localisationca': localisationca,
        'localisationco': localisationco,
        'reference': article_text  # Ajouter la colonne "reference"
    })

df = pd.DataFrame(data)

'''
article : nombre d’articles dans la source => `//article`
article_to_link : nombre d’articles à lier (en France) => `//article[not(localisationpa)]`
article_commune : nombre d’articles de type commune => `//article[not(localisationpa) and not(localisationco)]`
article_loc_com : nombre d’articles localisés dans une commune => `//article[not(localisationpa) and localisationco]`
'''
report = dict(
    article = i,
    article_to_link = len(df.index),
    article_commune = (df.localisationco == 'none').sum(),
    article_loc_com = (df.localisationco != 'none').sum()
)

# check
[print(k,':',v) for k, v in report.items()]
df.iloc[1500:1510, :]

article : 5178
article_to_link : 5178
article_commune : 2780
article_loc_com : 2398


,old-id,vedette,localisationde,localisationca,localisationco,reference
1500,PO11-01501,Fontaines,Vendée,none,none,Fontaines Vendée Fontanae prior ...
1501,PO11-01502,Fontaines-d’Ozillac,Charente-Maritime,none,none,Fontaines d Ozillac Charente Maritime...
1502,PO11-01503,Fontarède,Lot-et-Garonne,none,Caumont,Fontarède Lot et Garonne Caumont ...
1503,PO11-01504,Fontclaireau,Charente,none,none,Fontclaireau Charente Fons Claras...
1504,PO11-01505,Fontcouverte,Charente-Maritime,none,none,Fontcouverte Charente Maritime Sa...
1505,PO11-01506,Fontcreuse,Charente,none,Saint-Coutant,Fontcreuse Charente Saint Coutant...
1506,PO11-01507,Fontdouce,Charente-Maritime,none,Saint-Bris-des-Bois,Fontdouce Charente Maritime Saint...
1507,PO11-01508,Fontenay-le-Comte,Vendée,none,none,Fontenay le Comte Vendée Crucifix...
1508,PO11-01509,Fontenelles (Les),Vendée,none,La Roche-sur-Yon,Fontenelles Les Vendée La Roche...
1509,PO11-01510,Fontenet,Charente-Maritime,none,none,Fontenet Charente Maritime Fontan...


In [3]:
# Copiez les colonnes d'origine
df['vedette_orig'] = df['vedette']
df['localisationco_orig'] = df['localisationco']

In [4]:
#premier préprocessing pour les caractères spéciaux

In [5]:
def replace_special_characters(text):
    text = unidecode(text)
    text = re.sub(r'[-\'\[\]\(\)]', ' ', text)
    return text


df['vedette'] = df['vedette'].apply(replace_special_characters)
df['localisationco'] = df['localisationco'].apply(replace_special_characters)

In [6]:
df

,old-id,vedette,localisationde,localisationca,localisationco,reference,vedette_orig,localisationco_orig
0,PO11-00001,Abbaye L,Deux-Sèvres,none,Vasles,Abbaye L Deux Sèvres Vasles ...,Abbaye (L’),Vasles
1,PO11-00002,Abbaye L,Dordogne,none,Chatres,Abbaye L Dordogne Châtres ...,Abbaye (L’),Châtres
2,PO11-00003,Abbaye de Brignon L,Maine-et-Loire,none,Saint Macaire du Bois,Abbaye de Brignon L Maine et Loire...,Abbaye de Brignon (L’),Saint-Macaire-du-Bois
3,PO11-00004,Abbaye des Bois L,Deux-Sèvres,none,Secondigny,Abbaye des Bois L Deux Sèvres ...,Abbaye des Bois (L’),Secondigny
4,PO11-00005,Absie L,Deux-Sèvres,none,none,Absie L Deux Sèvres Abscia in ...,Absie (L’),none
...,...,...,...,...,...,...,...,...
5173,PO11-05174,Yves,Charente-Maritime,none,none,Yves Charente Maritime Sanctus St...,Yves,none
5174,PO11-05175,Yviers,Charente,none,none,Yviers Charente Sancta Maria d Yv...,Yviers,none
5175,PO11-05176,Yvrac,Charente,none,none,Yvrac Charente Yvracum 150 162 ...,Yvrac,none
5176,PO11-05177,Yvrac,Gironde,none,none,Yvrac Gironde Sanctus Vincencius ...,Yvrac,none


In [7]:
# Ajout du dpt code au df
dpt_df = pd.read_csv('../../utils/pouille/resources/departements-region.csv')
dpt_dict = dict(zip(dpt_df['dep_name'], dpt_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['dpt_code'] = df['localisationde'].apply(lambda x: dpt_dict.get(x, 'none'))

#on renomme et réordonne pour plus de clarté.
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune'})
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'reference', 'vedette_orig', 'localisationco_orig']]

df.iloc[1500:1510, :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
1500,PO11-01501,Fontaines,Vendée,85,none,none,Fontaines Vendée Fontanae prior ...,Fontaines,none
1501,PO11-01502,Fontaines d Ozillac,Charente-Maritime,17,none,none,Fontaines d Ozillac Charente Maritime...,Fontaines-d’Ozillac,none
1502,PO11-01503,Fontarede,Lot-et-Garonne,47,none,Caumont,Fontarède Lot et Garonne Caumont ...,Fontarède,Caumont
1503,PO11-01504,Fontclaireau,Charente,16,none,none,Fontclaireau Charente Fons Claras...,Fontclaireau,none
1504,PO11-01505,Fontcouverte,Charente-Maritime,17,none,none,Fontcouverte Charente Maritime Sa...,Fontcouverte,none
1505,PO11-01506,Fontcreuse,Charente,16,none,Saint Coutant,Fontcreuse Charente Saint Coutant...,Fontcreuse,Saint-Coutant
1506,PO11-01507,Fontdouce,Charente-Maritime,17,none,Saint Bris des Bois,Fontdouce Charente Maritime Saint...,Fontdouce,Saint-Bris-des-Bois
1507,PO11-01508,Fontenay le Comte,Vendée,85,none,none,Fontenay le Comte Vendée Crucifix...,Fontenay-le-Comte,none
1508,PO11-01509,Fontenelles Les,Vendée,85,none,La Roche sur Yon,Fontenelles Les Vendée La Roche...,Fontenelles (Les),La Roche-sur-Yon
1509,PO11-01510,Fontenet,Charente-Maritime,17,none,none,Fontenet Charente Maritime Fontan...,Fontenet,none


In [8]:
# Chargement du référentiel des communes
fields = ['insee_code', 'DEP_id', 'NCCENR']
main_insee_commune = pd.read_csv("../../utils/pouille/resources/main_insee_commune.tsv",
                                 delimiter='\t',
                                 usecols=fields,
                                 dtype={'insee_code': 'string'})
main_insee_commune['DEP_id'] = main_insee_commune['DEP_id'].apply(lambda x: x[4:])
main_insee_commune.head(5)

,insee_code,DEP_id,NCCENR
0,01001,01,Abergement-Clémenciat
1,01002,01,Abergement-de-Varey
2,01004,01,Ambérieu-en-Bugey
3,01005,01,Ambérieux-en-Dombes
4,01006,01,Ambléon


In [9]:
main_insee_commune['NCCENR_orig'] = main_insee_commune['NCCENR']

In [10]:
main_insee_commune['NCCENR'] = main_insee_commune['NCCENR'].apply(replace_special_characters)

In [11]:
main_insee_commune.head(5)

,insee_code,DEP_id,NCCENR,NCCENR_orig
0,01001,01,Abergement Clemenciat,Abergement-Clémenciat
1,01002,01,Abergement de Varey,Abergement-de-Varey
2,01004,01,Amberieu en Bugey,Ambérieu-en-Bugey
3,01005,01,Amberieux en Dombes,Ambérieux-en-Dombes
4,01006,01,Ambleon,Ambléon


In [12]:
# Liste les départements présents pour mieux filtrer quand pas de localisationde
dpt_list = df['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)

['12', '13', '15', '16', '17', '18', '19', '23', '24', '26', '31', '32', '33', '34', '35', '36', '37', '38', '40', '41', '43', '44', '46', '47', '49', '60', '62', '64', '71', '75', '77', '79', '82', '84', '85', '86', '87']


# Exact match methods

## Liage des articles de type commune (exact match mathod 1)

In [13]:
# dataframe à charger
linked_places_df = pd.DataFrame()

In [14]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
communes_df.iloc[500:1000, :]
#communes_df.shape

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
840,PO11-00841,Champcevinel,Dordogne,24,none,none,Champcevinel Dordogne Campus Savi...,Champcevinel,none
842,PO11-00843,Champdolent,Charente-Maritime,17,none,none,Champdolent Charente Maritime Cam...,Champdolent,none
847,PO11-00848,Champigny le Sec,Vienne,86,none,none,Champigny le Sec Vienne Campaygni...,Champigny-le-Sec,none
848,PO11-00849,Champigny sur Veude,Indre-et-Loire,37,none,none,Champigny sur Veude Indre et Loire ...,Champigny-sur-Veude,none
849,PO11-00850,Champmillon,Charente,16,none,none,Champmillon Charente Campusmillon...,Champmillon,none
...,...,...,...,...,...,...,...,...,...
1692,PO11-01693,Graulges Les,Dordogne,24,none,none,Graulges Les Dordogne Grangae ...,Graulges (Les),none
1703,PO11-01704,Grezac,Charente-Maritime,17,none,none,Grézac Charente Maritime Grassat ...,Grézac,none
1705,PO11-01706,Grezes,Dordogne,24,none,none,Grèzes Dordogne Gresae 595 ...,Grèzes,none
1707,PO11-01708,Grezillac,Gironde,33,none,none,Grézillac Gironde Gresilhacum pr...,Grézillac,none


### With dpt

In [15]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and dpt_code
communes_exact_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette', 'dpt_code'],
                                 right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_dpt_df['insee'] = communes_exact_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_dpt_df = communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_dpt_df = communes_exact_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
communes_exact_dpt_df['method'] = 'dpt_exact'

# Check for duplicate entries
duplicates = communes_exact_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_dpt_df = communes_exact_dpt_df[~duplicates]


# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, communes_exact_dpt_df]).drop_duplicates()

In [16]:
# Tests
#communes_exact_dpt_df.head(5)
communes_exact_dpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
0,PO11-00006,Abzac,Gironde,33,none,none,Abzac Gironde Sanctus Petrus d Ab...,Abzac,none,Abzac,Abzac (33001),dpt_exact
1,PO11-00007,Adilly,Deux-Sèvres,79,none,none,Adilly Deux Sèvres Azille 355,Adilly,none,Adilly,Adilly (79002),dpt_exact
2,PO11-00009,Adriers,Vienne,86,none,none,Adriers Vienne Adrer 427 446 ...,Adriers,none,Adriers,Adriers (86001),dpt_exact
3,PO11-00012,Agen,Lot-et-Garonne,47,none,none,Agen Lot et Garonne Agenensis Ag...,Agen,none,Agen,Agen (47001),dpt_exact
4,PO11-00014,Agme,Lot-et-Garonne,47,none,none,Agmé Lot et Garonne Accmerium pro...,Agmé,none,Agmé,Agme (47002),dpt_exact
...,...,...,...,...,...,...,...,...,...,...,...,...
2484,PO11-05171,Xambes,Charente,16,none,none,Xambes Charente Cembesium 170 ...,Xambes,none,Xambes,Xambes (16423),dpt_exact
2485,PO11-05174,Yves,Charente-Maritime,17,none,none,Yves Charente Maritime Sanctus St...,Yves,none,Yves,Yves (17483),dpt_exact
2486,PO11-05175,Yviers,Charente,16,none,none,Yviers Charente Sancta Maria d Yv...,Yviers,none,Yviers,Yviers (16424),dpt_exact
2487,PO11-05177,Yvrac,Gironde,33,none,none,Yvrac Gironde Sanctus Vincencius ...,Yvrac,none,Yvrac,Yvrac (33554),dpt_exact


In [17]:
# des doublons de liage exact match dans un même dpt ?
#communes_exact_dpt_df.article_id.duplicated().sum()
communes_exact_dpt_df.loc[communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method


### Without dpt

In [18]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and NCCENR
communes_exact_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_nodpt_df['insee'] = communes_exact_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_nodpt_df = communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_nodpt_df = communes_exact_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
communes_exact_nodpt_df['method'] = 'nodpt_exact'

duplicates = communes_exact_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_nodpt_df = communes_exact_nodpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, communes_exact_nodpt_df]).drop_duplicates()

In [19]:
# On cherche à savoir combien ont été liés sans information de département ; cela signifie que dans le XML,
# on a des communes qui ne sont pas localisées.
# Test
communes_exact_nodpt_df
#linked_places_df.shape
communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,3,3,1,1,1,1,3,3,1,3,3,1
count,3,3,3,3,3,3,3,3,3,3,3,3
size,3,3,3,3,3,3,3,3,3,3,3,3


In [20]:
# TODO: on teste quoi ?
communes_exact_nodpt_df.loc[communes_exact_nodpt_df.duplicated(keep=False,subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method


## Liage des communes de localisation (exact match method 1)

### With dpt

In [21]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
0,PO11-00001,Abbaye L,Deux-Sèvres,79,none,Vasles,Abbaye L Deux Sèvres Vasles ...,Abbaye (L’),Vasles
1,PO11-00002,Abbaye L,Dordogne,24,none,Chatres,Abbaye L Dordogne Châtres ...,Abbaye (L’),Châtres
2,PO11-00003,Abbaye de Brignon L,Maine-et-Loire,49,none,Saint Macaire du Bois,Abbaye de Brignon L Maine et Loire...,Abbaye de Brignon (L’),Saint-Macaire-du-Bois
3,PO11-00004,Abbaye des Bois L,Deux-Sèvres,79,none,Secondigny,Abbaye des Bois L Deux Sèvres ...,Abbaye des Bois (L’),Secondigny
9,PO11-00010,Agassas,Lot-et-Garonne,47,none,Labretonie,Agassas Lot et Garonne Labretonie...,Agassas,Labretonie
...,...,...,...,...,...,...,...,...,...
5161,PO11-05162,Voutron,Charente-Maritime,17,none,Yves,Voutron Charente Maritime Yves ...,Voutron,Yves
5165,PO11-05166,Vrille,Deux-Sèvres,79,none,Moutiers sous Argenton,Vrillé Deux Sèvres Moutiers sous ...,Vrillé,Moutiers-sous-Argenton
5166,PO11-05167,Vuhe,Charente-Maritime,17,none,La Jarne,Vuhe Charente Maritime La Jarne ...,Vuhe,La Jarne
5171,PO11-05172,Xandeville,Charente,16,none,Barbezieux Saint Hilaire,Xandeville Charente Barbezieux Sa...,Xandeville,Barbezieux-Saint-Hilaire


In [22]:
# Merge pas_communes_df and main_insee_commune on localisationco and dpt_code
pas_communes_exact_dpt_df = pd.merge(pas_communes_df,
                                     main_insee_commune,
                                     how='inner',
                                     left_on=['localisationco', 'dpt_code'],
                                     right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_dpt_df['insee'] = pas_communes_exact_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
pas_communes_exact_dpt_df['method'] = 'dpt_exact'

duplicates = pas_communes_exact_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_dpt_df = pas_communes_exact_dpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_dpt_df]).drop_duplicates()


In [23]:
# Tests
pas_communes_exact_dpt_df.head(5)
#pas_communes_exact_dpt_df.shape
pas_communes_exact_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,2175,1579,21,21,1,1018,2172,1581,1018,1018,1023,1
count,2175,2175,2175,2175,2175,2175,2175,2175,2175,2175,2175,2175
size,2175,2175,2175,2175,2175,2175,2175,2175,2175,2175,2175,2175


### Without dpt

In [24]:
import pandas as pd

# Merge pas_communes_df and main_insee_commune on localisationco and NCCENR
pas_communes_exact_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                       main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                       how='inner',
                                       left_on='localisationco',
                                       right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_nodpt_df['insee'] = pas_communes_exact_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
pas_communes_exact_nodpt_df['method'] = 'nodpt_exact'

duplicates = pas_communes_exact_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_nodpt_df]).drop_duplicates()

In [25]:
# On cherche à savoir combien ont été liés sans information de département ; cela signifie que dans le XML,
# on a des lieux dans des communes qui ne sont pas localisés.
# Test
pas_communes_exact_nodpt_df
#linked_places_df.shape
pas_communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,0,0,0,0,0,0,0,0,0,0,0,0
count,0,0,0,0,0,0,0,0,0,0,0,0
size,0,0,0,0,0,0,0,0,0,0,0,0


## Liage des articles de type commune (exact match method 2)

On cherche à ré-effectuer un deuxième liage en exact match pour tous les articles n'ayant pas été liés dans la première passe. Nous effectuons un pré-processing plus poussé ; on retire tous les mots d'une liste pré-définie par Olivier (tels que Le, La, L', ruisseau, bois, rivière, etc) sur les vedettes du pouillés + les NCCENR du référentiel INSEE puis on ré-applique notre script.

### Data preprocessing

In [26]:
file_path = '../../utils/pouille/resources/tokens_dicotopo.txt'

with open(file_path, 'r') as file:
    tokens = file.read() 

In [27]:
# filtrer les mots des noms de lieux
def filter_words(text, tokens):
    text = replace_special_characters(text)  # Appliquer la fonction replace_special_characters
    words = text.split()
    words = [word for word in words if not re.search(r'\b{}\b'.format(re.escape(word.lower())), tokens)]
    filtered_text = ' '.join(words).strip() 
    return filtered_text

df['vedette'] = df['vedette'].apply(lambda x: filter_words(x, tokens=tokens))
df['localisationco'] = df['localisationco'].apply(lambda x: filter_words(x, tokens=tokens))

In [28]:
main_insee_commune['NCCENR'] = main_insee_commune['NCCENR'].apply(lambda x: filter_words(x, tokens=tokens))

In [29]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
#communes_df.shape

### with dpt

In [30]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and dpt_code
communes_exact_simple_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette', 'dpt_code'],
                                 right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_simple_dpt_df['insee'] = communes_exact_simple_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_simple_dpt_df = communes_exact_simple_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_simple_dpt_df = communes_exact_simple_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
communes_exact_simple_dpt_df['method'] = 'dpt_exact_simple'

duplicates = communes_exact_simple_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_simple_dpt_df = communes_exact_simple_dpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = communes_exact_simple_dpt_df.loc[communes_exact_simple_dpt_df['method'] == 'dpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

### without dpt

In [31]:
# Merge communes_df and main_insee_commune on vedette and NCCENR
communes_exact_simple_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_simple_nodpt_df['insee'] = communes_exact_simple_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
communes_exact_simple_nodpt_df['method'] = 'nodpt_exact_simple'

duplicates = communes_exact_simple_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = communes_exact_simple_nodpt_df.loc[communes_exact_simple_nodpt_df['method'] == 'nodpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

## Liage des communes de localisation

In [32]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
0,PO11-00001,Abbaye,Deux-Sèvres,79,none,Vasles,Abbaye L Deux Sèvres Vasles ...,Abbaye (L’),Vasles
1,PO11-00002,Abbaye,Dordogne,24,none,Chatres,Abbaye L Dordogne Châtres ...,Abbaye (L’),Châtres
2,PO11-00003,Abbaye Brignon,Maine-et-Loire,49,none,Macaire,Abbaye de Brignon L Maine et Loire...,Abbaye de Brignon (L’),Saint-Macaire-du-Bois
3,PO11-00004,Abbaye,Deux-Sèvres,79,none,Secondigny,Abbaye des Bois L Deux Sèvres ...,Abbaye des Bois (L’),Secondigny
9,PO11-00010,Agassas,Lot-et-Garonne,47,none,Labretonie,Agassas Lot et Garonne Labretonie...,Agassas,Labretonie
...,...,...,...,...,...,...,...,...,...
5161,PO11-05162,Voutron,Charente-Maritime,17,none,Yves,Voutron Charente Maritime Yves ...,Voutron,Yves
5165,PO11-05166,Vrille,Deux-Sèvres,79,none,Moutiers Argenton,Vrillé Deux Sèvres Moutiers sous ...,Vrillé,Moutiers-sous-Argenton
5166,PO11-05167,Vuhe,Charente-Maritime,17,none,Jarne,Vuhe Charente Maritime La Jarne ...,Vuhe,La Jarne
5171,PO11-05172,Xandeville,Charente,16,none,Barbezieux Hilaire,Xandeville Charente Barbezieux Sa...,Xandeville,Barbezieux-Saint-Hilaire


### with dpt

In [33]:
# Merge pas_communes_df and main_insee_commune on localisationco and dpt_code
pas_communes_exact_simple_dpt_df = pd.merge(pas_communes_df,
                                     main_insee_commune,
                                     how='inner',
                                     left_on=['localisationco', 'dpt_code'],
                                     right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_simple_dpt_df['insee'] = pas_communes_exact_simple_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
pas_communes_exact_simple_dpt_df['method'] = 'dpt_exact_simple'

duplicates = pas_communes_exact_simple_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = pas_communes_exact_simple_dpt_df.loc[pas_communes_exact_simple_dpt_df['method'] == 'dpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

### without dpt

In [34]:
import pandas as pd

# Merge pas_communes_df and main_insee_commune on localisationco and NCCENR
pas_communes_exact_simple_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                       main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                       how='inner',
                                       left_on='localisationco',
                                       right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_simple_nodpt_df['insee'] = pas_communes_exact_simple_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
pas_communes_exact_simple_nodpt_df['method'] = 'nodpt_exact_simple'

duplicates = pas_communes_exact_simple_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = pas_communes_exact_simple_nodpt_df.loc[pas_communes_exact_simple_nodpt_df['method'] == 'nodpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

linked_places_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
0,PO11-00006,Abzac,Gironde,33,none,none,Abzac Gironde Sanctus Petrus d Ab...,Abzac,none,Abzac,Abzac (33001),dpt_exact
1,PO11-00007,Adilly,Deux-Sèvres,79,none,none,Adilly Deux Sèvres Azille 355,Adilly,none,Adilly,Adilly (79002),dpt_exact
2,PO11-00009,Adriers,Vienne,86,none,none,Adriers Vienne Adrer 427 446 ...,Adriers,none,Adriers,Adriers (86001),dpt_exact
3,PO11-00012,Agen,Lot-et-Garonne,47,none,none,Agen Lot et Garonne Agenensis Ag...,Agen,none,Agen,Agen (47001),dpt_exact
4,PO11-00014,Agme,Lot-et-Garonne,47,none,none,Agmé Lot et Garonne Accmerium pro...,Agmé,none,Agmé,Agme (47002),dpt_exact
...,...,...,...,...,...,...,...,...,...,...,...,...
2341,PO11-04871,Toumine,Charente-Maritime,17,none,Thou,Toumine Charente Maritime Le Thou...,Toumine,Le Thou,Thou,Thou (17447),dpt_exact_simple
2350,PO11-04958,Vandeloigne,Deux-Sèvres,79,none,Ferriere Parthenay,Vandeloigne Deux Sèvres La Ferriè...,Vandeloigne,La Ferrière-en-Parthenay,Ferrière-en-Parthenay,Ferriere Parthenay (79120),dpt_exact_simple
2351,PO11-04960,Vanneau,Deux-Sèvres,79,none,Vanneau Irleau,Vanneau Le Deux Sèvres Le Vanne...,Vanneau (Le),Le Vanneau-Irleau,Vanneau-Irleau,Vanneau Irleau (79337),dpt_exact_simple
2356,PO11-04981,Vaure,Charente,16,none,Pins,Vaure La Charente Les Pins ...,Vaure (La),Les Pins,Pins,Pins (16261),dpt_exact_simple


# Fuzzy match method

## Liage des articles de type commune (fuzzy method)

### With dpt

In [35]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les communes à lier avec le département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' n'est pas égale à 'none'.
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']

# Utiliser la colonne 'vedette' comme valeur pour la colonne 'insee_code'
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']

# Pour chaque ligne dans le DataFrame "communes_fuzzy_dpt_df"
for _, x in communes_fuzzy_dpt_df.iterrows():
    # Sélectionner les valeurs de la colonne 'NCCENR' du DataFrame "main_insee_commune" où la colonne 'DEP_id' correspond à la valeur de la colonne 'dpt_code' de la ligne en cours (x).
    dpt_nccenr = main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR']
    
    # Effectuer un fuzzy match entre les NCCENR du département et la valeur de la colonne 'vedette' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in dpt_nccenr
        if all(word in nccenr.split() for word in x['vedette'].split()) and fuzz.token_sort_ratio(nccenr, x['vedette']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    communes_fuzzy_dpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "communes_fuzzy_dpt_df" et définir toutes ses valeurs comme 'dpt_fuzzy'.
communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "communes_fuzzy_dpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_dpt_df]).drop_duplicates()

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_26648/786987469.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']


In [36]:
#méthode avec difflib

'''
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['vedette'],
            main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['vedette'].split())
    ],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_dpt_df]).drop_duplicates()
'''

'\nimport difflib\n\n# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE\nnccenr_to_insee = main_insee_commune.set_index(\'NCCENR\')[\'insee_code\'].to_dict()\n\n# Lier les communes avec dpt de localisation (dpt/fuzzy match)\ncommunes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df[\'article_id\'])][communes_df.dpt_code != \'none\']\ncommunes_fuzzy_dpt_df[\'insee_code\'] = communes_fuzzy_dpt_df[\'vedette\']\ncommunes_fuzzy_dpt_df[\'insee_code\'] = communes_fuzzy_dpt_df.apply(\n    lambda x: [\n        f"{nccenr} ({nccenr_to_insee.get(nccenr, \'\')}) - {main_insee_commune[main_insee_commune[\'NCCENR\'] == nccenr][\'NCCENR_orig\'].iloc[0]}"\n        for nccenr in difflib.get_close_matches(\n            x[\'vedette\'],\n            main_insee_commune[main_insee_commune[\'DEP_id\'] == x[\'dpt_code\']][\'NCCENR\'],\n            n=3, cutoff=0.1\n        )\n        if all(word in nccenr.split() for word in x[\'vedette\'].split())\n    ],\n   

In [37]:
# Tests
#communes_fuzzy_dpt_df.head(20)
#linked_places_df.shape
#linked_places_df
communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,18,17,6,6,1,1,18,18,1,17,1
count,18,18,18,18,18,18,18,18,18,18,18
size,18,18,18,18,18,18,18,18,18,18,18


In [38]:
communes_fuzzy_dpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
164,PO11-00165,Aubie,Gironde,33,none,none,Aubie Gironde Albia prior 40 5...,Aubie,none,Aubie et Espessas (33018) - Aubie-et-Espessas,dpt_fuzzy
569,PO11-00570,Bretonniere,Vendée,85,none,none,Bretonnière La Vendée Bertoneri...,Bretonnière (La),none,Bretonniere Claye (85036) - Bretonnière-la-Claye,dpt_fuzzy
956,PO11-00957,Chatelaillon,Charente-Maritime,17,none,none,Châtelaillon Charente Maritime Ca...,Châtelaillon,none,Chatelaillon Plage (17094) - Châtelaillon-Plage,dpt_fuzzy
1221,PO11-01222,Criteuil,Charente,16,none,none,Criteuil Charente Christolium 17...,Criteuil,none,Criteuil Magdeleine (16116) - Criteuil-la-Magd...,dpt_fuzzy
1237,PO11-01238,Cumond,Dordogne,24,none,none,Cumond Dordogne Cucmon Cucmont ...,Cumond,none,Antoine Cumond (24368) - Saint-Antoine-Cumond,dpt_fuzzy
2123,PO11-02124,Magnac Lavalette,Charente,16,none,none,Magnac Lavalette Charente Manhacu...,Magnac-Lavalette,none,Magnac Lavalette Villars (16198) - Magnac-Lava...,dpt_fuzzy
2390,PO11-02391,Monsempron,Lot-et-Garonne,47,none,none,Monsempron Lot et Garonne Mons So...,Monsempron,none,Monsempron Libos (47179) - Monsempron-Libos,dpt_fuzzy
2471,PO11-02472,Montravel,Dordogne,24,none,none,Montravel Dordogne Monrevel arc...,Montravel,none,Lamothe Montravel (24226) - Lamothe-Montravel,dpt_fuzzy
2853,PO11-02854,Plassac,Charente,16,none,none,Plassac Charente Plassiacum 158 ...,Plassac,none,Plassac Rouffiac (16263) - Plassac-Rouffiac,dpt_fuzzy
3035,PO11-03036,Re,Charente-Maritime,17,none,none,Ré île Charente Maritime Ré ...,Ré,none,"Ars Re (17019) - Ars-en-Ré, Plage Re (17051) -...",dpt_fuzzy


### Without dpt

In [39]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les communes sans département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' est égale à 'none'.
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']

# Utiliser la colonne 'vedette' comme valeur pour la colonne 'insee_code'
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']

# Pour chaque ligne dans le DataFrame "communes_fuzzy_nodpt_df"
for _, x in communes_fuzzy_nodpt_df.iterrows():
    # Effectuer un fuzzy match entre les NCCENR du DataFrame "main_insee_commune" et la valeur de la colonne 'vedette' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in main_insee_commune['NCCENR']
        if all(word in nccenr.split() for word in x['vedette'].split()) and fuzz.token_sort_ratio(nccenr, x['vedette']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    communes_fuzzy_nodpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "communes_fuzzy_nodpt_df" et définir toutes ses valeurs comme 'nodpt_fuzzy'.
communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "communes_fuzzy_nodpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_nodpt_df]).drop_duplicates()

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_26648/3219305585.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']


In [40]:
#liage avec difflib

'''
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['vedette'],
            main_insee_commune['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['vedette'].split())
    ],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_nodpt_df]).drop_duplicates()
'''

'\nimport difflib\n\n# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE\nnccenr_to_insee = main_insee_commune.set_index(\'NCCENR\')[\'insee_code\'].to_dict()\n\n# Lier les communes sans dpt de localisation (fuzzy match)\ncommunes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df[\'article_id\'])][communes_df.dpt_code == \'none\']\ncommunes_fuzzy_nodpt_df[\'insee_code\'] = communes_fuzzy_nodpt_df[\'vedette\']\ncommunes_fuzzy_nodpt_df[\'insee_code\'] = communes_fuzzy_nodpt_df.apply(\n    lambda x: [\n        f"{nccenr} ({nccenr_to_insee.get(nccenr, \'\')}) - {main_insee_commune[main_insee_commune[\'NCCENR\'] == nccenr][\'NCCENR_orig\'].iloc[0]}"\n        for nccenr in difflib.get_close_matches(\n            x[\'vedette\'],\n            main_insee_commune[\'NCCENR\'],\n            n=3, cutoff=0.1\n        )\n        if all(word in nccenr.split() for word in x[\'vedette\'].split())\n    ],\n    axis=1\n)\n\n# Sortir les communes non liées

In [41]:
# Tests
communes_fuzzy_nodpt_df
#communes_fuzzy_nodpt_df.shape
#linked_places_df.shape
communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,22,22,2,1,1,1,22,22,1,22,1
count,22,22,22,22,22,22,22,22,22,22,22
size,22,22,22,22,22,22,22,22,22,22,22


In [42]:
communes_fuzzy_nodpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
182,PO11-00183,Aunis,none,none,none,none,Aunis Alniciensis Alnisiensis archi...,Aunis,none,Aigrefeuille Aunis (17003) - Aigrefeuille-d'Au...,nodpt_fuzzy
353,PO11-00354,Berry,none,none,none,none,Berry province Berri 224,Berry,none,"Berry Bac (02073) - Berry-au-Bac, Christophe a...",nodpt_fuzzy
612,PO11-00613,Bruilhois,none,none,none,none,Bruilhois Bruelhesium archipresbiter...,Bruilhois,none,Colombe Bruilhois (47238) - Sainte-Colombe-en-...,nodpt_fuzzy
618,PO11-00619,Buch,none,none,none,none,Buch Bogensis synodus 10 Bogiu...,Buch,none,Teste Buch (33529) - Teste-de-Buch,nodpt_fuzzy
925,PO11-00926,Chartreux,none,none,none,none,Chartreux Cartusiani ordo 67,Chartreux,none,Saulx Chartreux (91587) - Saulx-les-Chartreux,nodpt_fuzzy
1045,PO11-01046,Citeaux,none,none,none,none,Cîteaux Cisterciensis ordo 265 303...,Cîteaux,none,"Bessey Citeaux (21067) - Bessey-lès-Cîteaux, C...",nodpt_fuzzy
1298,PO11-01299,Dordogne,none,none,none,none,Dordogne La Dordonia rivière aff...,Dordogne (La),none,Beaulieu Dordogne (19019) - Beaulieu-sur-Dordo...,nodpt_fuzzy
1301,PO11-01302,Double,none,none,none,none,Double La Dupla Dupla sive Avanxen...,Double (La),none,"Andre Double (24367) - Saint-André-de-Double, ...",nodpt_fuzzy
1564,PO11-01565,Fronsadais,none,none,none,none,Fronsadais Fronsadesium archipresbit...,Fronsadais,none,Cadillac Fronsadais (33082) - Cadillac-en-Fron...,nodpt_fuzzy
1738,PO11-01739,Guyenne,none,none,none,none,Guyenne Guyenne huissier présidiaux...,Guyenne,none,Sauveterre Guyenne (33506) - Sauveterre-de-Guy...,nodpt_fuzzy


## Liage des communes de localisation (fuzzy method)

### With dpt

In [43]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les lieux dans les communes avec département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "pas_communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' n'est pas égale à 'none'.
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']

# Utiliser la colonne 'localisationco' comme valeur pour la colonne 'insee_code'
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']

# Pour chaque ligne dans le DataFrame "pas_communes_fuzzy_dpt_df"
for _, x in pas_communes_fuzzy_dpt_df.iterrows():
    # Sélectionner les valeurs de la colonne 'NCCENR' du DataFrame "main_insee_commune" où la colonne 'DEP_id' correspond à la valeur de la colonne 'dpt_code' de la ligne en cours (x).
    dpt_nccenr = main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR']
    
    # Effectuer un fuzzy match entre les NCCENR du département et la valeur de la colonne 'localisationco' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in dpt_nccenr
        if all(word in nccenr.split() for word in x['localisationco'].split()) and fuzz.token_sort_ratio(nccenr, x['localisationco']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    pas_communes_fuzzy_dpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "pas_communes_fuzzy_dpt_df" et définir toutes ses valeurs comme 'dpt_fuzzy'.
pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "pas_communes_fuzzy_dpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_26648/3940713726.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']


In [44]:
#liage avec difflib

'''

import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['localisationco'],
            main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['localisationco'].split())
    ],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()
'''

'\n\nimport difflib\n\n# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE\nnccenr_to_insee = main_insee_commune.set_index(\'NCCENR\')[\'insee_code\'].to_dict()\n\n# Lier les communes avec dpt de localisation (dpt/fuzzy match)\npas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df[\'article_id\'])][pas_communes_df.dpt_code != \'none\']\npas_communes_fuzzy_dpt_df[\'insee_code\'] = pas_communes_fuzzy_dpt_df[\'localisationco\']\npas_communes_fuzzy_dpt_df[\'insee_code\'] = pas_communes_fuzzy_dpt_df.apply(\n    lambda x: [\n        f"{nccenr} ({nccenr_to_insee.get(nccenr, \'\')}) - {main_insee_commune[main_insee_commune[\'NCCENR\'] == nccenr][\'NCCENR_orig\'].iloc[0]}"\n        for nccenr in difflib.get_close_matches(\n            x[\'localisationco\'],\n            main_insee_commune[main_insee_commune[\'DEP_id\'] == x[\'dpt_code\']][\'NCCENR\'],\n            n=3, cutoff=0.1\n        )\n        if all(word in nccenr.split() 

In [45]:
# Tests
pas_communes_fuzzy_dpt_df
#linked_places_df.shape
#linked_places_df
pas_communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,16,16,5,5,1,14,16,16,14,13,1
count,16,16,16,16,16,16,16,16,16,16,16
size,16,16,16,16,16,16,16,16,16,16,16


In [46]:
pas_communes_fuzzy_dpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
126,PO11-00127,Argentine,Dordogne,24,none,Rochebeaucourt,Argentine Dordogne La Rochebeauco...,Argentine,La Rochebeaucourt,Rochebeaucourt et Argentine (24353) - Rochebea...,dpt_fuzzy
286,PO11-00287,Beaulieu,Vendée,85,none,Mareuil Lay,Beaulieu Vendée Mareuil sur Lay ...,Beaulieu,Mareuil-sur-Lay,Mareuil Lay Dissais (85135) - Mareuil-sur-Lay-...,dpt_fuzzy
481,PO11-00482,Bost,Dordogne,24,none,Argentine,Bost Le Dordogne Argentine ...,Bost (Le),Argentine,Rochebeaucourt et Argentine (24353) - Rochebea...,dpt_fuzzy
866,PO11-00867,Chantrezac,Charente,16,none,Roumazieres,Chantrezac Charente Roumazières ...,Chantrezac,Roumazières,Roumazieres Loubert (16192) - Roumazières-Loubert,dpt_fuzzy
1083,PO11-01084,Cogulet,Charente,16,none,Vitrac,Cogulet Charente Vitrac Cogul...,Cogulet,Vitrac,Vitrac Vincent (16416) - Vitrac-Saint-Vincent,dpt_fuzzy
1359,PO11-01360,Epeluche,Dordogne,24,none,Comberanche,Épeluche Dordogne Comberanche ...,Épeluche,Comberanche,Comberanche et Epeluche (24128) - Comberanche-...,dpt_fuzzy
1477,PO11-01478,Fleurignac,Charente,16,none,Taponnat,Fleurignac Charente Taponnat ...,Fleurignac,Taponnat,Taponnat Fleurignac (16379) - Taponnat-Fleurignac,dpt_fuzzy
1502,PO11-01503,Fontarede,Lot-et-Garonne,47,none,Caumont,Fontarède Lot et Garonne Caumont ...,Fontarède,Caumont,Caumont Garonne (47061) - Caumont-sur-Garonne,dpt_fuzzy
2331,PO11-02332,Mirand,Dordogne,24,none,Cumond,Mirand Dordogne Cumond Miran ...,Mirand,Cumond,Antoine Cumond (24368) - Saint-Antoine-Cumond,dpt_fuzzy
3000,PO11-03001,Puyperoux,Charente,16,none,Aignes,Puypéroux Charente Aignes Pod...,Puypéroux,Aignes,Aignes et Puyperoux (16004) - Aignes-et-Puypéroux,dpt_fuzzy


### Without dpt

In [47]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les lieux dans les communes sans département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "pas_communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' est égale à 'none'.
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']

# Utiliser la colonne 'localisationco' comme valeur pour la colonne 'insee_code'
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']

# Pour chaque ligne dans le DataFrame "pas_communes_fuzzy_nodpt_df"
for _, x in pas_communes_fuzzy_nodpt_df.iterrows():
    # Effectuer un fuzzy match entre les NCCENR du DataFrame "main_insee_commune" et la valeur de la colonne 'localisationco' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in main_insee_commune['NCCENR']
        if all(word in nccenr.split() for word in x['localisationco'].split()) and fuzz.token_sort_ratio(nccenr, x['localisationco']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    pas_communes_fuzzy_nodpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "pas_communes_fuzzy_nodpt_df" et définir toutes ses valeurs comme 'nodpt_fuzzy'.
pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "pas_communes_fuzzy_nodpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_26648/895078486.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']


In [48]:
'''
#liage avec difflib

import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['localisationco'],
            main_insee_commune['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['localisationco'].split())
    ],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()
'''

'\n#liage avec difflib\n\nimport difflib\n\n# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE\nnccenr_to_insee = main_insee_commune.set_index(\'NCCENR\')[\'insee_code\'].to_dict()\n\n# Lier les communes sans dpt de localisation (fuzzy match)\npas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df[\'article_id\'])][pas_communes_df.dpt_code == \'none\']\npas_communes_fuzzy_nodpt_df[\'insee_code\'] = pas_communes_fuzzy_nodpt_df[\'localisationco\']\npas_communes_fuzzy_nodpt_df[\'insee_code\'] = pas_communes_fuzzy_nodpt_df.apply(\n    lambda x: [\n        f"{nccenr} ({nccenr_to_insee.get(nccenr, \'\')}) - {main_insee_commune[main_insee_commune[\'NCCENR\'] == nccenr][\'NCCENR_orig\'].iloc[0]}"\n        for nccenr in difflib.get_close_matches(\n            x[\'localisationco\'],\n            main_insee_commune[\'NCCENR\'],\n            n=3, cutoff=0.1\n        )\n        if all(word in nccenr.split() for word in x[\'localisa

In [49]:
pas_communes_fuzzy_nodpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method


In [50]:
# Tests
pas_communes_fuzzy_nodpt_df
#linked_places_df.shape
pas_communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,0,0,0,0,0,0,0,0,0,0,0
count,0,0,0,0,0,0,0,0,0,0,0
size,0,0,0,0,0,0,0,0,0,0,0


# Postprocessing

## Rapport et analyse

In [52]:
# On crée un DataFrame new_rows_df en filtrant les lignes de df qui ne sont pas présentes dans linked_places_df
new_rows_df = df[~df['article_id'].isin(linked_places_df['article_id'])]

# On vérifie si 'method' n'est pas dans les colonnes de new_rows_df
if 'method' not in new_rows_df.columns:
    new_rows_df['method'] = 'nulle'

# On utilise la méthode .concat pour ajouter les lignes de new_rows_df à linked_places_df
linked_places_df = pd.concat([linked_places_df, new_rows_df], ignore_index=True)

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_26648/297743551.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_rows_df['method'] = 'nulle'


In [53]:
linked_places_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
0,PO11-00006,Abzac,Gironde,33,none,none,Abzac Gironde Sanctus Petrus d Ab...,Abzac,none,Abzac,Abzac (33001),dpt_exact
1,PO11-00007,Adilly,Deux-Sèvres,79,none,none,Adilly Deux Sèvres Azille 355,Adilly,none,Adilly,Adilly (79002),dpt_exact
2,PO11-00009,Adriers,Vienne,86,none,none,Adriers Vienne Adrer 427 446 ...,Adriers,none,Adriers,Adriers (86001),dpt_exact
3,PO11-00012,Agen,Lot-et-Garonne,47,none,none,Agen Lot et Garonne Agenensis Ag...,Agen,none,Agen,Agen (47001),dpt_exact
4,PO11-00014,Agme,Lot-et-Garonne,47,none,none,Agmé Lot et Garonne Accmerium pro...,Agmé,none,Agmé,Agme (47002),dpt_exact
...,...,...,...,...,...,...,...,...,...,...,...,...
5173,PO11-04998,Vendeuvre Poitou,Vendée,85,none,none,Vendeuvre du Poitou Vendée Sancta...,Vendeuvre-du-Poitou,none,NaN,NaN,nulle
5174,PO11-05141,Vitre,Deux-Sèvres,79,none,Beaussais Vitre,Vitré Deux Sèvres Beaussais Vitré...,Vitré,Beaussais-Vitré,NaN,NaN,nulle
5175,PO11-05158,Voultegon,Deux-Sèvres,79,none,Voulmentin,Voultegon Deux Sèvres Voulmentin ...,Voultegon,Voulmentin,NaN,NaN,nulle
5176,PO11-05164,Vouzailles,Vendée,85,none,none,Vouzailles Vendée Vosaillia prio...,Vouzailles,none,NaN,NaN,nulle


In [54]:
linked_places_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,5178,4279,42,38,1,1133,5174,4251,1133,2872,3020,6
count,5178,5178,5178,5178,5178,5178,5178,5178,5178,5064,5120,5178
size,5178,5178,5178,5178,5178,5178,5178,5178,5178,5178,5178,5178


In [55]:
# des doublons?
nombre_de_doublons = linked_places_df.duplicated(subset=['article_id']).sum()
print(nombre_de_doublons)


0


In [56]:
linked_places_df.loc[linked_places_df.duplicated(keep=False,subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method


In [57]:
method_counts = linked_places_df['method'].value_counts()
print(method_counts)

method
dpt_exact           4662
dpt_exact_simple     399
nulle                 58
dpt_fuzzy             34
nodpt_fuzzy           22
nodpt_exact            3
Name: count, dtype: int64


In [58]:
vedette_list = pd.unique(linked_places_df['vedette'])
linked_places_df['sort_key'] = linked_places_df.apply(lambda x: x['localisationco'] if x['localisationco'] in vedette_list else x['vedette'], axis=1)
linked_places_df.sort_values(['method', 'sort_key', 'article_id'], inplace=True)
linked_places_df = linked_places_df.drop('sort_key', axis=1)
linked_places_df = linked_places_df[['article_id', 'vedette', 'vedette_orig', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'localisationco_orig', 'NCCENR_orig', 'insee_code', 'method', 'reference']]
linked_places_df

,article_id,vedette,vedette_orig,localisationde,dpt_code,canton_code,localisationco,localisationco_orig,NCCENR_orig,insee_code,method,reference
0,PO11-00006,Abzac,Abzac,Gironde,33,none,none,none,Abzac,Abzac (33001),dpt_exact,Abzac Gironde Sanctus Petrus d Ab...
1,PO11-00007,Adilly,Adilly,Deux-Sèvres,79,none,none,none,Adilly,Adilly (79002),dpt_exact,Adilly Deux Sèvres Azille 355
2,PO11-00009,Adriers,Adriers,Vienne,86,none,none,none,Adriers,Adriers (86001),dpt_exact,Adriers Vienne Adrer 427 446 ...
3528,PO11-01353,Entrefin,Entrefin,Vienne,86,none,Adriers,Adriers,Adriers,Adriers (86001),dpt_exact,Entrefin Vienne Adriers Entre...
3,PO11-00012,Agen,Agen,Lot-et-Garonne,47,none,none,none,Agen,Agen (47001),dpt_exact,Agen Lot et Garonne Agenensis Ag...
...,...,...,...,...,...,...,...,...,...,...,...,...
5134,PO11-01255,Dandesigny,Dandesigny,Vendée,85,none,Verrue,Verrue,NaN,NaN,nulle,Dandesigny Vendée Verrue Addo...
5174,PO11-05141,Vitre,Vitré,Deux-Sèvres,79,none,Beaussais Vitre,Beaussais-Vitré,NaN,NaN,nulle,Vitré Deux Sèvres Beaussais Vitré...
5175,PO11-05158,Voultegon,Voultegon,Deux-Sèvres,79,none,Voulmentin,Voulmentin,NaN,NaN,nulle,Voultegon Deux Sèvres Voulmentin ...
5176,PO11-05164,Vouzailles,Vouzailles,Vendée,85,none,none,none,NaN,NaN,nulle,Vouzailles Vendée Vosaillia prio...


In [59]:
empty_rows = linked_places_df[linked_places_df['vedette'] == '']
empty_rows

,article_id,vedette,vedette_orig,localisationde,dpt_code,canton_code,localisationco,localisationco_orig,NCCENR_orig,insee_code,method,reference
4979,PO11-01498,,Fontaine,Vendée,85,none,Bernard,Le Bernard,Bernard,Bernard (85022),dpt_exact_simple,Fontaine Vendée Le Bernard Fo...
4969,PO11-01748,,Haut Mont (Le),Charente-Maritime,17,none,Genetouze,La Genétouze,Genétouze,Genetouze (17173),dpt_exact_simple,Haut Mont Le Charente Maritime ...


In [60]:
empty_rows.agg(['nunique', 'count', 'size'])

,article_id,vedette,vedette_orig,localisationde,dpt_code,canton_code,localisationco,localisationco_orig,NCCENR_orig,insee_code,method,reference
nunique,2,1,2,2,2,1,2,2,2,2,1,2
count,2,2,2,2,2,2,2,2,2,2,2,2
size,2,2,2,2,2,2,2,2,2,2,2,2


## Export

Suite à tous nos traitements, nous exportons le résultat sous la forme d'un fichier CSV. 

Il prend cette forme;

article_id: id du pouillé

vedette: vedette du pouillé, traitée (donc sans ponctuation ni mots jugés 'dérangeants' au liage)

vedette_orig: vedette au format originel

localisationde: département

dpt_code: code du département

canton_code: canton

localisationco: pour les lieux dans des communes, localisationco du pouillé, traité (donc sans ponctuation ni mots jugés 'dérangeants' au liage)

localisationco_orig: localisationco au format originel

nccenr_orig: le nccenr originel du référentiel insee (uniquement pour les exact matches)

insee_code: le code insee. Pour les fuzzy, il y a le nccenr originel + nettoyé + le code insee pour chaque proposition; pour faciliter la correction d'Olivier

method: la méthode effectuée. Il y en a 7 en principe (à part si nous avons une absence de résultat pour la méthode): dpt_exact, nodpt_exact (methode exact match 1), dpt_exact_simple, no_dpt_exact_simple (method exact match 2), dpt_fuzzy, nodpt_fuzzy, nulle (lorsque nous n'avons pas pu faire le liage)

reference: tout le texte de chaque article pouillé


In [61]:
# Sauvegarder le dataframe fusionné
linked_places_df.to_csv('../../utils/pouille/out/linking_out/liage_po11.csv', index=False)


# Note – réflexions sur le fuzzy

Après avoir effectué des tests, on se rend compte qu'il y a plusieurs incohérences.

1. Difflib, avec main_insee_commune, n'arrive pas à lier certains cas en fuzzy (par exemple, Deneuille & Deneuille-lès-Chantelle). Pourtant, difflib y arrive lorsqu'il utilise un fichier txt de base. Peut-être un souci au niveau de main_insee_commune ? A verifier.

2. Au contraire, TheFuzz arrive à lier dans les deux cas: super. Par contre, il lie des choses en plus (Hures) avec main_insee_commune, mais pas avec le fichier txt. A verifier sur main_insee_commune: il ne semble pourtant pas y avoir d'espace ou de lettres en plus ou en moins.

Pour l'instant, nous gardons thefuzz, mais nous sauvegardons tous les codes de difflib en bas du notebook au cas où, comme les deux manières de faire semblent un peu faillibles (en attendant de comprendre).